In [ ]:
import numpy as np
import cv2
import os

 
images_folder = './N2-embryo-10um-SRx5-crop-images-correct-angle-folders'
seg_folder = './N2-embryo-10um-SRx5-crop-seg-correct-angle-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-embryo-10um-SRx5-3D-pointcloud"


try:
    os.makedirs(pointcloud_folder)
    print(f"Folder '{pointcloud_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{pointcloud_folder}' already exists.")   



In [ ]:
import math
import open3d
from copy import deepcopy
import csv

pi = np.pi
fx_scale = 5.0 



for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    
    
    

    
    
    print(sub_folder)

    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>350:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)

    
    cloud_points = None
    degree_per_frame = 360.0/(best_match_index+1)

    two_d_physical_parameters = []

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:

                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                img = cv2.imread(images_list_dir + '/' + image_name)

                h,w = mask.shape


                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                    center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                    area = cv2.contourArea(contours[i])
                    if area > 5000:
                        c_index = i

                
                widths = []
                y_positions = []


                
                _, y, w, h = cv2.boundingRect(contours[c_index])

                
                for i in range(h):
                    
                    ys = contours[c_index][:, 0, 1]
                    xs = contours[c_index][:, 0, 0]
                    row_inds = np.where(ys == (y + i))
                    min_x, max_x = np.min(xs[row_inds]), np.max(xs[row_inds])


                    
                    center = (min_x, y+i)  
                    radius = 1  
                    color = (0, 0, 255)  
                    
                    cv2.circle(img, center, radius, color, -1)  

                    
                    center = (max_x, y+i)  
                    radius = 1  
                    color = (255, 0, 0)  
                    
                    cv2.circle(img, center, radius, color, -1)  

                    cv2.line(img, (min_x, y+i), (max_x, y+i), (0, 255, 0), 1)

                    
                    
                    width = (max_x - min_x)/fx_scale

                    widths.append(width)
                    y_positions.append(y + i)

                
                widths = np.array(widths)
                radii = widths / 2
                y_positions = np.array(y_positions)

                
                surface_areas = 2 * pi * radii  
                volume_elements = pi * (radii ** 2)  

                
                total_surface_area = np.sum(surface_areas)/fx_scale
                total_volume = np.sum(volume_elements)/fx_scale

                total_surface_area, total_volume

                

                
                two_d_physical_parameters.append([image_index,total_surface_area,total_volume,h/fx_scale,w/fx_scale,h/w])

    
    csv_file = pointcloud_folder+'/'+sub_folder+"_2D physical_parameters.csv"

    
    header = ["image_index(1pixel=0.5um)", "surface area", "volume", "length", "width", "ratio"]

    
    with open(csv_file, "w", newline="") as file:
        
        csv_writer = csv.writer(file)
        
        
        csv_writer.writerow(header)
        
        
        csv_writer.writerows(two_d_physical_parameters)

    print(f"{csv_file} ！")
                

In [ ]:
print(two_d_physical_parameters)